In [ ]:
# pip install openai 
# pip install azure-search-documents==11.4.0b8

import openai
from openai.embeddings_utils import get_embedding
import json

# Azure OpenAI Credentials
OPENAI_API_KEY = "" 
OPENAI_RESOURCE_ENDPOINT = ""

# Azure Cognitive Search Credentials
ACS_ENDPOINT = ""
ACS_KEY = ""

openai.api_type = "azure"
openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_RESOURCE_ENDPOINT
openai.api_version = "2023-07-01-preview"

## Load the Hotel data and generate embeddings

In [ ]:
file_path = "hotels.json"

with open(file_path, 'r') as file:  
    text = file.read()  

hotels = json.loads(text)

for hotel in hotels:
    print(hotel['hotelname'])

    text_for_embedding = "Name: "+hotel['hotelname']+" " \
            +"Price min: "+str(hotel['pricemin'])+" euro max: "+str(hotel['pricemax'])+" euro " \
            +"Location: City: "+hotel['city']+" Country: "+hotel['country']+" " \
            +"Features of the hotel: "+hotel['features']+" " \
            +"Description: "+hotel['description']
    hotel['text'] = text_for_embedding
    hotel['textVector'] = get_embedding(text_for_embedding, engine = 'text-embedding-ada-002')
    hotel['featuresVector'] = get_embedding(hotel['features'], engine = 'text-embedding-ada-002')

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector

from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        HnswVectorSearchAlgorithmConfiguration,
        CorsOptions
)

In [ ]:
index_name = "hotels"

def get_index(name: str):

    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, filterable=True),
        SearchableField(name="hotelname", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="country", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="city", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="location", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="features", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="description", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="text", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SimpleField(name="pricemin", type=SearchFieldDataType.Int32,retrievable=True,searchable=True,filterable=True),
        SimpleField(name="pricemax", type=SearchFieldDataType.Int32,retrievable=True,searchable=True,filterable=True),
        SearchField(
            name="textVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_configuration="my-vector-config",
        ),   
        SearchField(
            name="featuresVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_configuration="my-vector-config",
        ), 
    ]

    cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
    scoring_profiles = []
    vector_search = VectorSearch(
        algorithm_configurations=[HnswVectorSearchAlgorithmConfiguration(name="my-vector-config", kind="hnsw")]
    )

    return SearchIndex(
            name=name, 
            fields=fields, 
            scoring_profiles=scoring_profiles, 
            vector_search=vector_search, 
            cors_options=cors_options)

client = SearchIndexClient(ACS_ENDPOINT, AzureKeyCredential(ACS_KEY))

index = get_index(index_name)
client.delete_index(index_name)
client.create_index(index)

search_client = SearchClient(ACS_ENDPOINT, index_name, AzureKeyCredential(ACS_KEY))
search_client.upload_documents(documents=hotels)

In [ ]:
ACS_INDEX_NAME = "hotels"

def simple_hybrid_search(query):
    search_client = SearchClient(ACS_ENDPOINT, ACS_INDEX_NAME, AzureKeyCredential(ACS_KEY))
    vector = Vector(value=get_embedding(query, engine = 'text-embedding-ada-002'), k=3, fields="textVector")

    results = search_client.search(
        search_text=query,
        vectors=[vector],
        select=["id", "hotelname","text","features"],
        top=5
    )
    
    return results

search_results = simple_hybrid_search(query = "Hotels with a private beach cost max 300 euro")

for result in search_results:
    print(result)


In [ ]:
def getAnswerFromOpenAI(query):

    # Search in the Azure Search Index
    azure_search_results = simple_hybrid_search(query)
    
    results_for_prompt=""
    for result in azure_search_results:
        results_for_prompt += f"Hotel: {result['text']}\n"
      
    system_message = f"""Assistant is a large language model designed to help users find Hotels
    You have access to an Azure Cognitive Search index with hundreds of hotels.

    This information returned from the search to anwser the users question
    {results_for_prompt}

    If you don't know the answer, just say that you don't know. Don't try to make up an answer.

    Give your response as a story, not a list.
    """
    
    response = openai.ChatCompletion.create(
        engine="gpt-4-0613", #"gpt-35-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": query},
        ]
    )

    return response['choices'][0]['message']['content']

result = getAnswerFromOpenAI("Hotels with a private beach cost max 300 euro")
print(result)

## Azure OpenAI Function Calling

In [ ]:
ACS_INDEX_NAME = "hotels"

def search_hotels(query, location, max_price, features):
 
    search_client = SearchClient(ACS_ENDPOINT, ACS_INDEX_NAME, AzureKeyCredential(ACS_KEY))
    vector_text = Vector(value=get_embedding(query, engine = 'text-embedding-ada-002'), k=3, fields="textVector")
    vector_features = Vector(value=get_embedding(features, engine = 'text-embedding-ada-002'), k=3, fields="featuresVector")

    results = search_client.search(  
        search_text=None,  
        vectors=[vector_text,vector_features],
        filter=f"pricemin lt {max_price}",
        select=["id", "hotelname","text","features","pricemax","pricemin"],
        top=5
    )  

    for result in results:
        print("--")
        print(result['hotelname'])
        print("Min",result['pricemin'],"Max",result['pricemax'])
        print(result['features'])
        print(result['@search.score'])
        
    return results

# Test the method
search_hotels("Hotel with a private beach cost max 300 euro in Delmaris.","Delmaris",300,"private beach, free wifi, yoga")

In [ ]:
# Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai

# 1 – Call the chat completions API with your functions and the user’s input

messages= [
    {"role": "system", "content": "You're an AI assistant designed to help users search for hotels. When a user asks for help finding a hotel, you should call the search_hotels function."},
    {"role": "system", "content": "Don't make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "Hotel with a private beach cost max 300 euro in Delmaris."}
]

functions= [  
    {
        "name": "search_hotels",
        "description": "Retrieves hotels from the search index based on the parameters provided",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The query string to search for a hotel",
                },
                "location": {
                    "type": "string",
                    "description": "The location of the hotel (i.e. Seattle, WA)"
                },
                "max_price": {
                    "type": "number",
                    "description": "The maximum price for the hotel"
                },
                "features": {
                    "type": "string",
                    "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)"
                }
            },
            "required": ["query","location","max_price","features"]
        }
    }
]  

response = openai.ChatCompletion.create(
    deployment_id="gpt-35-turbo",
    messages=messages,
    functions=functions,
    function_call="auto", 
)
response_message = response["choices"][0]["message"]

print("Step 1 - Response:")
print(response_message)

#
# 2 – Use the model’s response to call your API or function
#

# Check if the model wants to call a function
if response_message.get("function_call"):

    # Call the function. The JSON response may not always be valid so make sure to handle errors
    function_name = response_message["function_call"]["name"]

    available_functions = {
            "search_hotels": search_hotels,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)

    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message["role"],
            "function_call": {
                "name": function_name,
                "arguments": response_message["function_call"]["arguments"],
            },
            "content": None
        }
    )

    # Adding the search results to the prompt
    for result in function_response:
        results_for_prompt = f"Hotel: {result['text']}\n"    
        messages.append( # adding function response to messages
            {
                "role": "function",
                "name": function_name,
                "content": results_for_prompt,
            }
        ) 

    # Call the API again to get the final response from the model
    second_response = openai.ChatCompletion.create(
        messages = messages,
        deployment_id = "gpt-35-turbo"
    )
    
    print("----\nStep 2 - Response:")
    print(second_response["choices"][0]["message"]['content'])
else:
    print(response["choices"][0]["message"])